Start this first
```bash
$ litex_server uart /dev/ttyUSB0 1152000
```

In [1]:
%pylab notebook
from litex.soc.tools.remote import RemoteClient
from struct import pack, unpack
import time

Populating the interactive namespace from numpy and matplotlib


In [23]:
for i in range(32):
    try:
        r = RemoteClient(csr_csv="build/csr.csv", debug=False, port=1234 + i)
        r.open()
        print("Connected to Port", 1234 + i)
        break
    except ConnectionRefusedError:
        pass

def getId():
    s = ""
    for i in range(64):
        temp = r.read(r.bases.identifier_mem + i * 4)
        if temp == 0:
            break
        s += chr(temp & 0xFF)
    return s
getId()

Connected to Port 1234


'SP605 eth demo 2019-04-08 19:14:13'

In [24]:
r.regs.ethphy_mode_detection_mode.read()

0

In [6]:
r.regs.ethphy_crg_reset.write(0)

In [19]:
r.regs.ethphy_mdio_r.read()

1

In [18]:
r.regs.ethphy_mdio_w.write(0)

In [25]:
from litescope.software.driver.analyzer import LiteScopeAnalyzerDriver
wb = r

In [27]:
analyzer = LiteScopeAnalyzerDriver(wb.regs, "analyzer", config_csv="build/analyzer.csv", debug=True)

In [45]:
analyzer.configure_subsampler(1)
analyzer.add_rising_edge_trigger("icmp_echo_sink_sink_ready")
analyzer.run(offset=128, length=256)
analyzer.wait_done()

[running]...


In [48]:
analyzer.upload()
analyzer.save("dump.vcd")

[uploading]...
|====================>| 100%
[writing to dump.vcd]...


In [ ]:
r.close()